In [1]:
%%capture
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import numpy as np
import h5py

In [2]:
from tensorflow.keras.models import load_model
from utils import read_h5_dataset
from BilinearInterpolate3D import BilinearInterpolate3D
from OrthogonalConv import NetXCycle

In [3]:
BATCH_SIZE = 64

## Load CIFAR10 model

In [4]:
%%capture
cifar_model_path = 'model_val_accuracy.h5'
cifar_dataset_path = 'test_00001.h5'
cifar_model = load_model(cifar_model_path, custom_objects={'BilinearInterpolate3D':BilinearInterpolate3D})

In [5]:
# Compiling the model
opt_rms = RMSprop(lr=0.001,decay=1e-6)
cifar_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Load dataset
x_test, y_test = read_h5_dataset(cifar_dataset_path)

cifar_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 80)        2240      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32, 32, 80)        320       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 80)        0         
_________________________________________________________________
bilinear_interpolate3d (Bili (None, 75, 75, 75)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 75)        50700     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 75, 75, 75)        300 

In [9]:
# Evaluating 
scores = cifar_model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)

10000/10000 [==============================] - 9s 898us/sample - loss: 0.4911 - acc: 0.8815


## Load PATHFINDER model

In [7]:
# Specify the model directory path - no trailing `/`
pathfinder_model_dir = 'model_kernel20_path9'

In [8]:
!sh untar.sh $pathfinder_model_dir

!ls model_kernel20_path9/

model_kernel20_path9.h5
model.tar.gz.partaa  model.tar.gz.partab  model_kernel20_path9.h5


In [9]:
%%capture
pathfinder_model_path = pathfinder_model_dir + '/' + pathfinder_model_dir + '.h5'

pathfinder_model = load_model(pathfinder_model_path, custom_objects={'BilinearInterpolate3D':BilinearInterpolate3D,
                                                                        'NetXCycle': NetXCycle})

!rm $pathfinder_model_path

In [10]:
pathfinder_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 128)     51328     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 128, 128, 128)     512       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 128)     0         
_________________________________________________________________
net_x_cycle (NetXCycle)      (None, 128, 128, 128)     19662720  
_________________________________________________________________
global_max_pooling2d (Global (None, 128)               0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0     